In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import os
import json
import cv2
import numpy as np
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import to_categorical

In [9]:
NUM_CLASSES = 4
IMG_WIDTH, IMG_HEIGHT = 224, 224 # VGG16 모델의 이미지 입력 크기
BATCH_SIZE = 32

In [17]:
# JSON 파일이 있는 폴더 경로
json_folder_path = r'C:\label'

# 이미지 파일이 있는 폴더 경로
image_folder_path = r'C:\img'

In [18]:
# 클래스별 이미지와 레이블을 저장할 리스트
train_images = []
train_labels = []
val_images = []
val_labels = []

emotions = ["happy", "anger", "panic", "sadness"]

In [19]:
def preprocess_image(image_path, image_name):
    try:
        image = cv2.imread(image_path) # 이미지를 읽어옴
        if image is None:
            raise ValueError(f"Image {image_name} could not be read.")
        image = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT)) # 이미지 리사이징
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = preprocess_input(image) # 모델의 입력에 맞게 이미지 전처리
        return image
    except Exception as e:
        print(f"Error processing image {image_name}: {e}")
        return None


In [20]:
def process_data(emotion, data_type):
    images = []
    labels = []
    json_file_path = os.path.join(json_folder_path, data_type, f"{data_type}_{emotion}.json") # 각 감정의 json 파일 경로
    image_file_path = os.path.join(image_folder_path, data_type, f"{emotion}") # 각 감정의 이미지 파일 경로

    with open(json_file_path, 'r', encoding='euc-kr') as f:
        data = json.load(f)

    if not data:
        print(f"JSON 파일에 데이터가 없습니다: {json_file_path}")

    for item in data:
        image_name = item["filename"]
        image_path = os.path.join(image_file_path, image_name)

        if not os.path.exists(image_path):
            print(f"not found {image_path}.")
            continue

        vgg16_input = preprocess_image(image_path, image_name)
        if vgg16_input is None:
            continue

        images.append(vgg16_input)
        labels.append(emotion)

    return images, labels

In [21]:
for emotion in emotions:
    train_images_emotion, train_labels_emotion = process_data(emotion, 'train')
    train_images.extend(train_images_emotion)
    train_labels.extend(train_labels_emotion)

for emotion in emotions:
    val_images_emotion, val_labels_emotion = process_data(emotion, 'val')
    val_images.extend(val_images_emotion)
    val_labels.extend(val_labels_emotion)

not found C:\img\train\happy\7zwecfe6e57f975ee8b238b355e3e87923c4557383daae96720c1b8c3f669g9m2.jpg.
not found C:\img\train\happy\oxmt62fbb28fafcdaa8d64e75a332e1d506d6a079cb45fc9e00c576c40fbeo0wy.jpg.
not found C:\img\train\happy\s1mu862ea97f26b2720eeecb51156f6e32e8523fd62f3f04ef2f727639f52d7g3.jpg.
not found C:\img\train\panic\twdva8d723865fa1005873a6967797c75f3dd83197f13f7daa3e3926947071al6.jpg.
not found C:\img\train\panic\5yid6d7ae32bf14b1343da46cfdb3fdffc8e1c64baf4fb1b175690f648a870v69.jpg.
not found C:\img\train\panic\bpv531e5cebdb1c00085158bb0677c5596ccabf1c8a5cb0007991571bf306wcd4.jpg.
not found C:\img\train\panic\s1q397a38900c1e1f950496109900e7c01f0630459790d8590010655c4090vlh0.jpg.
not found C:\img\train\panic\7m2k46c05803855399ea1ee1911875ee59e05640c93135c0bc91b5660456cdo8k.jpg.
not found C:\img\train\panic\n55yc741f09701506c7eb5ba30533db35c905ab5bf0597b5652ece03c5950ozv1.jpg.
not found C:\img\train\panic\bid7b3c93409e1d83514689337f6d3d63a338339d71c9ce7fded153d3576fk2ml.jpg.


In [22]:
# numpy 배열로 변환
train_images, train_labels = np.array(train_images), np.array(train_labels)
val_images, val_labels = np.array(val_images), np.array(val_labels)

In [23]:
#Label Encoding (레이블 인코딩) : 문자열 레이블(감정)을 정수로 변환
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)

In [24]:
# to_categorical :정수 인코딩된 레이블을 받아들이고 원-핫 인코딩된 레이블을 반환
train_labels = to_categorical(train_labels_encoded, NUM_CLASSES)
val_labels = to_categorical(val_labels_encoded, NUM_CLASSES)

In [25]:
# VGG16 모델 불러오기
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [26]:
# 기존 레이어의 가중치 동결
for layer in base_model.layers:
    layer.trainable = False

In [27]:
# 새로운 모델 레이어 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [28]:
# 새로운 모델 구성
model = Model(inputs=base_model.input, outputs=predictions)

In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │         4,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,244,100 (58.15 MB)

 Trainable params: 529,412 (2.02 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [30]:
# 모델 훈련
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels), batch_size=32)

Epoch 1/10
168/168 ━━━━━━━━━━━━━━━━━━━━ 1492s 9s/step - accuracy: 0.2851 - loss: 3.5357 - val_accuracy: 0.3346 - val_loss: 1.3704
Epoch 2/10
161/168 ━━━━━━━━━━━━━━━━━━━━ 46s 7s/step - accuracy: 0.3877 - loss: 1.3006

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(val_images, val_labels)
print("정확도:", accuracy)

In [ ]:
average_accuracy = np.mean(history.history['val_accuracy'])
print("평균 검증 정확도:", average_accuracy)